##Installs & Imports


In [14]:
!pip install -U bokeh seaborn pandas
!pip install git+https://github.com/FordyceLab/seequence.git#egg=seequence
!pip install primer3-py biopython pandarallel
!pip install pulp
!pip install gurobipy
!pip install biopython==1.75


  Cloning https://github.com/FordyceLab/seequence.git to /tmp/pip-install-yo3pmvda/seequence_afab25e774ab4a28a861bf7cb6c991ee
  Running command git clone --filter=blob:none --quiet https://github.com/FordyceLab/seequence.git /tmp/pip-install-yo3pmvda/seequence_afab25e774ab4a28a861bf7cb6c991ee
  Resolved https://github.com/FordyceLab/seequence.git to commit 3ea730537fcf5b7ef807ebf6e057f5bf4e875bb9
  Preparing metadata (setup.py) ... done


In [15]:
# IMPORTS
import time

import random as rand
from itertools import product

from seequence.view import *
from seequence.color import *

from pandarallel import pandarallel as pl
pl.initialize()

import primer3 as p3
from Bio.Seq import Seq
from Bio.SeqUtils import GC, seq1, seq3
from Bio.SeqUtils.CodonUsage import SharpEcoliIndex, SynonymousCodons

import itertools as it
import numpy as np
import pandas as pd
pd.set_option('display.precision', 1)
import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.models.annotations import Span
from bokeh.models import Select
from bokeh.layouts import column
from IPython.display import display, clear_output

import gurobipy as gp

import pickle

import tracemalloc
import matplotlib.pyplot as plt

def revcomp(seq):
  return str(Seq(seq).reverse_complement())
def translate(seq):
  return str(Seq(seq).translate())

# clear_output()
print('Ready')

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Ready


## Find primer sequences

In [16]:
def n_subsequences(sequence, lmin, lmax):
  print(sum(len(sequence) - l + 1 for l in range(lmin, lmax+1)))

def subsequences(sequence, lmin, lmax): #Generates all subsequences w/ all poss. start-stop pairs
  ls = []
  for j in range(lmin, lmax+1): #length
    for i in range(len(sequence)-j+1): #starting index
      start = i
      stop = i+j
      ls.append([sequence[start:stop], start, stop, stop-start])
  return pd.DataFrame(ls, columns=['seq','start','stop','len'])

## Primer Data
##### Creates the primer pandas dataframe containing information about every possible primer in the protein coding sequence. The function calculates various primer parameters including gc content, delta G and melting temepratures. The cost is also calculated for every primer

In [17]:
def create_primer_df(sequence_nt):
  # convention: start index of r-primers will be 3' (i.e. start < stop)
  primer_f = pd.DataFrame(columns=['seq','start','stop','fr','len'])
  primer_f[['seq','start','stop','len']] = subsequences(sequence_nt, primer_lmin, primer_lmax)
  primer_f['fr'] = 'f'

  #Shifting so that 0 is at the start of mutreg (upstream has negative values)
  primer_f['start'] = primer_f.start - len(upstream_nt)
  primer_f['stop'] = primer_f.stop - len(upstream_nt)

  #Creating reverse primers at same locations
  primer_r = primer_f[['seq','start','stop','fr','len']].copy()
  primer_r['fr'] = 'r'
  primer_r['seq'] = primer_r.seq.apply(revcomp)

  #Concatenating Forward & Reverse
  primer_df = pd.concat([primer_f,primer_r])
  primer_df.sort_values(by=['start','stop','fr'], inplace=True)

  #Calculating "Cost" Values
  primer_df['gc'] = primer_df.seq.apply(GC)
  primer_df['tm'] = primer_df.seq.apply(pcr.calcTm)
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
  primer_df['hp_tm'] = res.apply(lambda res: res['tm'])
  primer_df['hp_dg'] = res.apply(lambda res: res['dg']*1e-3)
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHomodimer(s).todict())
  primer_df['ho_tm'] = res.apply(lambda res: res['tm'])
  primer_df['ho_dg'] = res.apply(lambda res: res['dg']*1e-3)

  def primer_cost(primer):
    # calculates primer cost based on homodimer an haipin delta G, tm cost, and len cost
    hp_dg_max = -4
    ho_dg_max = -8
    tm_min = 58

    tm_cost = max(0, tm_min-primer.tm)**1.7
    hp_cost = max(0, hp_dg_max - primer.hp_dg)**1.2
    ho_cost = max(0, ho_dg_max - primer.ho_dg)**1.2
    len_cost = primer.len*1e-5

    cost = hp_cost + ho_cost + len_cost + tm_cost
    return cost

  primer_df['cost'] = primer_df.parallel_apply(primer_cost, axis=1)
  primer_df['log10cost'] = primer_df.cost.apply(np.log10)

  primer_df.reset_index(inplace=True)
  primer_f = primer_df.query('fr=="f"').reset_index(drop=True)
  primer_r = primer_df.query('fr=="r"').reset_index(drop=True)
  primer_df.set_index(['start','stop','fr'], inplace=True)

  return primer_f, primer_df

def check_threshold(tm,gc):
  # returns false only if the threshold flag is on and primers did not pass threshold
  if not apply_threshold:
    return True
  else:
    if min_gc <= gc <= max_gc and  min_tm <= tm <=max_tm:
      return True
    else:
      return False



## Primer Class
##### Creates a primer class which is used for the creation of the primer graph

In [18]:
class Primer:
  def __init__(self,primer_df,start,stop,is_r=False):
    assert start < stop
    self.start = start
    self.stop = stop
    self.is_r = is_r #forward or reverse
    self.l = stop-start #length
    self.primer_df = primer_df
    self.w = primer_df.at[self.tup(),'cost'] #total cost value; the fancy notation is b/c
                                             #of the hierarchal lookup system in panda.df

  def __str__(self):
    return ' '.join(map(str,(self.start, self.stop, self.is_r)))
  def __repr__(self):
    return f'{("r" if self.is_r else"f")}({self.start},{self.stop})'
  def tup(self):
    return (self.start,self.stop,("r" if self.is_r else"f"))


def actions(primer_df,primer): #returning possible counterparts (forward -> reverse; reverse -> forward)
                    #i.e. this method gets the "neighbors"
  if not primer.is_r:  # fwd
    for oligo_l, primer_l in it.product(reversed(range(oligo_lmin, oligo_lmax+1)),
                                        range(primer_lmin,primer_lmax+1)):

      stop = primer.start + oligo_l
      start = stop - primer_l

      if apply_threshold:
        # finds tm of forward and reverse primers
        tm_f=primer_df.at[primer.tup(),'tm']
        tm_r=primer_df.at[(start,stop,"r"),'tm']

        # if tm difference is larger then max_difference threshold do not add primer to graph
        if abs(tm_f-tm_r)>max_difference:
          continue

      yield Primer(primer_df,start, stop, is_r=True)

  elif primer.is_r:  # rev
    for overlap_l, primer_l in it.product(reversed(range(overlap_lmin, overlap_lmax+1)),
                                          range(primer_lmin,primer_lmax+1)):

      start = primer.stop - overlap_l
      stop = start + primer_l


      # filter
      no_split = (primer.start - stop) >= primer.start%3
      if (stop > primer.start) or (not no_split): ## redundant to check first condition?
        continue
      yield Primer(primer_df,start, stop)

## DFS for graph creation

In [19]:
def dfs(G, primer, primer_df, mutreg_l): #CREATING the graph

  tm=primer_df.at[primer.tup(),'tm']
  gc=primer_df.at[primer.tup(),'gc']

  if (primer.start >= mutreg_l) and primer.is_r and check_threshold(tm,gc) :  # base case (end)
    G.add_edge(primer.tup(),'d', weight=0.) #G is global variable defined in next section
    return

  for next_primer in actions(primer_df,primer):

    is_new = not G.has_node(next_primer.tup()) # check if primer node exists already

    tm=primer_df.at[next_primer.tup(),'tm']
    gc=primer_df.at[next_primer.tup(),'gc']

    passes_threshold= check_threshold(tm,gc) # check if primer passes threshold

    # only add edge if primer passes threshold
    if passes_threshold:
      G.add_edge(primer.tup(),next_primer.tup(), weight=next_primer.w) #weight is the cost of the new primer
    if passes_threshold and is_new:
      dfs(G,next_primer,primer_df,mutreg_l)

def paths_ct(G, u, d): #total # of paths between two points
    if u == d:
        return 1
    else:
        if not G.nodes[u]: #npaths attribute is the # of paths out of u
            G.nodes[u]['npaths'] = sum(paths_ct(G, c, d) for c in G.successors(u))
        return G.nodes[u]['npaths']

## Greedy algorithm implementation

In [20]:
def run_greedy(sequences_nt, mutreg_regions):
    path_ls = []
    total_cost = 0

    for seq_nt, mutreg_nt in zip(sequences_nt, mutreg_regions):

        print("Protein number:", len(path_ls))

        G, primer_df = create_single_graph(mutreg_nt, seq_nt)

        G_sub = G.copy()

        found_path = False

        while not found_path:

            nodes_to_remove = []

            # Find the shortest path, excluding the start ('s') and end ('d') nodes
            shortest_path = nx.algorithms.shortest_path(G_sub, 's', 'd', weight='weight')[1:-1]

            for primer in shortest_path:
                start, end, fr = primer

                # Calculate the primer sequence
                primer_seq = seq_nt[start+mutreg_start:end+mutreg_start]  # Assuming start and end are relative to the sequence

                cross_hybridization = False

                for i, path in enumerate(path_ls):

                    other_seq = sequences_nt[i]

                    for other_primer in path:
                        other_start, other_end, other_fr = other_primer
                        other_primer_seq = other_seq[other_start+mutreg_start:other_end+mutreg_start]

                        tm = p3.bindings.calc_heterodimer(other_primer_seq, primer_seq, mv_conc=50.0, dv_conc=1.5, dntp_conc=0.6,
                                                          dna_conc=50.0, temp_c=37.0, max_loop=30).tm
                        if tm >= 45:
                            cross_hybridization = True
                            break

                    if cross_hybridization:
                        break

                if cross_hybridization:
                    nodes_to_remove.append(primer)

            if nodes_to_remove:
                G_sub.remove_nodes_from(set(nodes_to_remove))
            else:
                found_path = True
                path_ls.append(shortest_path)
                print(shortest_path)

                # Calculate the cost for the found path
                primer_set = primer_df.loc[[p for p in shortest_path]].copy().reset_index()
                primer_cost = primer_set['cost'].sum()
                total_cost += primer_cost

    return path_ls, total_cost

## Gurobi setup

In [21]:
def get_model():
  params = {
  "WLSACCESSID":"96998216-eecd-4c3d-a975-2e0444b290f2",
  "WLSSECRET":"40d56b13-61f6-4ce4-aac2-e07498f76065",
  "LICENSEID":2499075
  }
  env = gp.Env(params=params)

  # Create the model within the Gurobi environment
  model = gp.Model('min-sum', env=env)
  return model

##Full Sequences
##### The user can choose the mutreg sequence and upstream and downstream regions

In [22]:
# constant upstream and downstream regions for 10 proteins
upstream_nt = 'GCTAGTGGTGCTAGCCCCGCGAAATTAATACGACTCTCTATAGGGTCTAGAAATAATTTTCTTTAACTTTAAGAAGGAGATATACAT'
downstream_nt = 'GGAGGGTCTGGGGCAGGAGGCAGTGGCATGGTGACCAAGGGCGTGGAGCTGTTCACCGGGGTGGTGCCCATCCTGGTCGAGCTGGACGGCGACGTAAACGGCCACAAGTTCAGCGTGTCCGGCGAGGGCGAGGGCGATGCCACCTACGGCAAGCTGACCCTGAAGTTCATCTGCACCACCGGCAAGCTGCCCG'

mutreg_regions=[]
protein_names=[]

# read 10 protein coding sequences from file
with open('10 coding sequences.txt') as file:
  for line in file.readlines():
    p_name, mutreg_region = line.strip().split('\t')
    mutreg_regions.append(mutreg_region)
    protein_names.append(p_name)

sequences_nt=[]

for mutreg_nt in mutreg_regions:
  sequence= upstream_nt + mutreg_nt + downstream_nt
  sequences_nt.append(sequence)

## Define parameters

In [23]:
# adjust algorithm parameters
primer_lmin, primer_lmax = 18, 30
overlap_lmin,overlap_lmax = 45,50
oligo_lmin,oligo_lmax = 195,205
allowed_overlap = 6
mutreg_start = len(upstream_nt)

# sets up pcr reaction with parameters
pcr = p3.thermoanalysis.ThermoAnalysis(dna_conc= 250,
                                       mv_conc= 50,
                                       dv_conc= 0,
                                       dntp_conc= 0,
                                       tm_method= 'santalucia',
                                       salt_correction_method= 'owczarzy',
                                       temp_c= 25)

## Efficiency Thresholds
##### The user can choose to apply primer efficiency threshold on gc content and tm and the maximum allowed tm different between forward and reverse primer in each pair

In [24]:
apply_threshold= False # apply threshold flag. If true, threshold will be applied
min_gc=40
max_gc=60
min_tm=58
max_tm=65
max_difference=3

## Create primer graph

In [25]:
def create_single_graph(mutreg_nt,sequence_nt):

    primer_f, primer_df = create_primer_df(sequence_nt)

    mutreg_l = len(mutreg_nt)

    G = nx.DiGraph()

    primers_init = [Primer(primer_df,p.start,p.stop) for _,p in primer_f.query('stop<=0').iterrows() if check_threshold(p.tm,p.gc)]  ## all forward primers that end before mutreg and pass threshold
    for primer in primers_init:
      G.add_edge('s',primer.tup(), weight=primer.w) #intializing the s-primer connection
      dfs(G, primer,primer_df,mutreg_l) #create the rest of the graph

    return G, primer_df

## Run greedy algorithm

In [26]:

start_time = time.time()
greedy_solution, greedy_obj = run_greedy(sequences_nt, mutreg_regions)
greedy_time = time.time() - start_time

all_data=[]

all_data.append({"Greedy Solution": greedy_solution,
                "Greedy Objective": greedy_obj,
                "Greedy Time": greedy_time})

print(all_data[-1])

run_df = pd.DataFrame(all_data)

# Write the DataFrame to a CSV file
run_df.to_csv('run_greedy.csv', index=False)

Protein number: 0


/usr/local/lib/python3.10/dist-packages/pandas/core/algorithms.py:1743: UserWarning: Function deprecated please use "calc_tm" instead
  return lib.map_infer(values, mapper, convert=convert)
<ipython-input-17-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" instead
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
<ipython-input-17-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" instead
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
<ipython-input-17-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" instead
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
<ipython-input-17-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" instead
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
<ipython-input-17-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin"

[(-83, -65, 'f'), (104, 122, 'r')]
Protein number: 1


/usr/local/lib/python3.10/dist-packages/pandas/core/algorithms.py:1743: UserWarning: Function deprecated please use "calc_tm" instead
  return lib.map_infer(values, mapper, convert=convert)
<ipython-input-17-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" instead
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
<ipython-input-17-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" instead
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
<ipython-input-17-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" instead
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
<ipython-input-17-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin" instead
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
<ipython-input-17-0f795d5c67af>:23: UserWarning: Function deprecated please use "calc_hairpin"

[(-83, -65, 'f'), (104, 122, 'r')]
{'Greedy Solution': [[(-83, -65, 'f'), (104, 122, 'r')], [(-83, -65, 'f'), (104, 122, 'r')]], 'Greedy Objective': 0.00072, 'Greedy Time': 23.369624614715576}
